|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating neurons and dimensions<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Context-modulated activation in MLP<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import requests
import textwrap

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Exercise 1: Import the model and the nouns, implant the hook

In [ ]:
# Eleuther's tokenizer
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125m')

# load in GPTneo's and push to GPU
model = AutoModelForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# move the model to the GPU
model = model.to(device)
model.eval();

# number of MLP 'expansion' units
nneurons =

In [ ]:
# a hook function to grab the activations
activations = {}

def implant_hook(layer_number):
  def hook(module, input, output):

    # get the activations
    acts =

    # store in the dictionary
    activations[f'mlp_{layer_number}_x'] =
  return hook


# pick the layer to hook
layer2hook = 8
hookName =

# surgery ;)
model.transformer.h[layer2hook].mlp.

In [ ]:
# import the nouns
url = 'https://raw.githubusercontent.com/david47k/top-english-wordlists/refs/heads/master/top_english_nouns_lower_10000.txt'
nouns = requests.get(url).text
nouns =

# Exercise 2: Activations for words with and without spaces

In [ ]:
# initialize tensor for all activations
all_activations = np.zeros()


# loop over the tokens
for i,word

  # forward pass one token without space
  with torch.no_grad(): model
  all_activations[0,i,:] = activations

  # forward pass same token with preceeding space
  with torch.no_grad(): model(
  all_activations[1,i,:] = activations


In [ ]:
# extract vectorized (flattened) variables
noSpace  =
yesSpace =

# square root of distances
diffs =
diffs /= # normalize by max-val


# scatter plots
_,axs = plt.subplots(1,2,figsize=(12,5))
axs[0].scatter(,,s=50,c=diffs,alpha=.5,cmap=mpl.cm.plasma_r)
axs[0].set(xlabel='No space',ylabel='With space',
              title=f'Correlation r = {}')


# histograms
y_yes,x_yes = np.histogram(,bins=100,density=
y_noS,x_noS = np.histogram(,bins=100,density=

axs[1].plot(x_yes[:-1],y_yes,linewidth=2,label='With space')


axs[1].legend()


plt.tight_layout()
plt.show()

# Exercise 3: Get activations from generated tokens

In [ ]:
# generate some new tokens
gentoks = model.generate(tokenizer.encode('I think the world could be better if',return_tensors='pt').to(device),
                         max_length=200, do_sample=True)

# let's see what the model thinks :o
print(

In [ ]:
with torch.no_grad(): model(gentoks)
activations[hookName].shape

In [ ]:
fulltext_activations = activations[hookName].cpu()

# Exercise 4: Compare in-text and individual tokens

In [ ]:
allacts = np.zeros((2,nneurons,len(gentoks[0])))

for ti,tok in

  # forward pass for just this token
  with torch.no_grad(): model(tok.unsqueeze(0).to(device))

  # get the two activations
  allacts[0,:,ti] = fulltext_activations
  allacts[1,:,ti] = activations


In [ ]:
# flattened activations again
fromText =
fromToks =

# normalized difference magnitudes
diffs =
diffs /=


# scatter plots
_,axs = plt.subplots(1,2,figsize=(12,5))


# histograms
y_toks,x_toks = np.histogram
y_text,x_text = np.histogram
y_diff,x_diff = np.histogram

axs[1].plot(label='Individual tokens')
axs[1].plot(label='From text')
axs[1].plot(label='Difference')

axs[1].legend()
axs[1].set(xlim=[-7,7],xlabel='Activation value',ylabel='Density',title='Distributions of MLP activations')

plt.tight_layout()
plt.show()